In [1]:
import numpy as np
import os
import cv2 as cv

from constants import *
from hsv_sliders import *
from image_processing import *
from visualisation import *
from solution import *


In [2]:
# get files 

files = os.listdir(INPUT_DIR)

solutions = sorted([file for file in files if os.path.splitext(file)[1] == ".txt"])
photos = sorted([file for file in files if os.path.splitext(file)[1] == ".jpg"])

# print(photos)


In [3]:
# show template board and all squares
tpl = cv.imread(TEMPLATE_PATH)

board = get_trimmed(tpl)
board = process_board(board)

# show_squares(board)


In [4]:
# test 2 photos
templates = load_templates()

idx = 164
prev_board = get_board(photos[idx - 1])
board = get_board(photos[idx])

move_pos = get_board_change(prev_board, board)
res_string = str(BOARD_Y_VALS[move_pos[Y]]) + str(BOARD_X_VALS[move_pos[X]])

flt_board = process_board(board)
current_square = process_square(get_square(flt_board, move_pos[X], move_pos[Y]))
# plt.imshow(current_square)
# plt.show()
# plt.imshow(templates[28])
# plt.show()
# plt.imshow(templates[25])
# plt.show()
# print(get_similitude(current_square, templates))

move_val = get_similitude(process_square(get_square(flt_board, move_pos[X], move_pos[Y])), templates)
res_string = res_string + " " + str(move_val)

print(res_string, "\t", end = "\n")



11G 28 	


In [5]:
prev_board = get_board(EMPTY_BOARD_PATH)
templates = load_templates()

offset = 3 * 2

start_idx = 0 + int(50 * offset / 2)
stop_idx = 50 + int(50 * offset / 2)

idx = start_idx + offset

for photo in photos[start_idx:stop_idx]: 
    # photo = photos[49]
    board = get_board(photo)
    move_pos = get_board_change(prev_board, board)
    res_string = str(BOARD_Y_VALS[move_pos[Y]]) + str(BOARD_X_VALS[move_pos[X]])

    flt_board = process_board(board)
    move_val = get_similitude(process_square(get_square(flt_board, move_pos[X], move_pos[Y])), templates)
    res_string = res_string + " " + str(move_val)

    with open(os.path.join(INPUT_DIR, solutions[idx]), 'r') as file:
        first_line = file.readline()
    print(res_string == first_line, "\t", "pos: ", idx - offset, "\t", sep = "", end = "\t")
    print(res_string, "\t", first_line, end = "\n")

    prev_board = board
    idx += 1

# templates = load_templates()

# show_squares(flt_board)
# show_img(flt_board)

# hsv_sliders(flt_board, RESIZE_SHOW, RESIZE_SHOW)


True	pos: 150		7I 3 	 7I 3
True	pos: 151		7J 6 	 7J 6
True	pos: 152		6H 6 	 6H 6
True	pos: 153		5H 8 	 5H 8
True	pos: 154		9G 4 	 9G 4
True	pos: 155		8I 1 	 8I 1
True	pos: 156		8J 5 	 8J 5
True	pos: 157		7K 9 	 7K 9
True	pos: 158		8K 5 	 8K 5
True	pos: 159		4H 48 	 4H 48
True	pos: 160		3H 40 	 3H 40
True	pos: 161		2H 8 	 2H 8
True	pos: 162		1H 32 	 1H 32
True	pos: 163		10G 7 	 10G 7
True	pos: 164		11G 28 	 11G 28
True	pos: 165		12G 21 	 12G 21
True	pos: 166		13G 7 	 13G 7
True	pos: 167		14G 3 	 14G 3
True	pos: 168		9H 6 	 9H 6
True	pos: 169		10H 24 	 10H 24
True	pos: 170		11H 30 	 11H 30
True	pos: 171		12H 54 	 12H 54
True	pos: 172		9K 45 	 9K 45
True	pos: 173		8L 25 	 8L 25
True	pos: 174		8M 5 	 8M 5
True	pos: 175		8N 5 	 8N 5
True	pos: 176		7L 3 	 7L 3
True	pos: 177		7M 12 	 7M 12
True	pos: 178		7N 15 	 7N 15
True	pos: 179		9J 1 	 9J 1
True	pos: 180		10J 6 	 10J 6
True	pos: 181		6J 11 	 6J 11
True	pos: 182		5J 5 	 5J 5
True	pos: 183		4J 16 	 4J 16
True	pos: 184		10F 17 	 10F 17
True	